In [1]:
import numpy as np 
from sklearn.linear_model import LinearRegression 
import torch 
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset , TensorDataset ,DataLoader 
from torch.utils.data.dataset import random_split 
from torch.utils.tensorboard import SummaryWriter


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use('fivethirtyeight')

In [4]:
%run -i data_preparation/v0.py
%run  -i model_configuration/v0.py

In [5]:
# %load model_training/v0.py

torch.manual_seed(42)
n_epochs = 100 
for epoch in range(n_epochs):
    #set the model to train
    model.train()

    #compute the model prediction
    yhat = model(x_train_tensor)

    #step-2 loss 
    loss = loss_fn(yhat , y_train_tensor)

    #compute gradients 
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()


In [6]:
#higher order function 
# means it does either 2 things , takes function as inputs or  return function
def exponentiation_builder(exponent):
    def skeleton_exponentiation(x):
        return x ** exponent

    return skeleton_exponentiation
        

In [10]:
square = exponentiation_builder(2)
cube = exponentiation_builder(3)
square(3)

9

In [12]:
cube(3)

27

In [13]:
def make_train_step(model , loss_fn , optimizer):
    def perform_train_step_fn(x ,y ):
        model.train()

        ###### model prediction
        yhat = model(x_train_tensor)

    ########## loss
        loss = loss_fn(yhat , y)

        #compute gradient
        loss.backward()

        ###### upgrade parameters
        optimizer.step()
        optimizer.zero_grad()

        return loss.item()

    return perform_train_fn